In [9]:
!docker-compose up -d --scale node=3

Creating network "k8s_default" with the default driver
Creating volume "k8s_k3s-server" with default driver
Creating k8s_server_1 ... 
ting k8s_server_1 ... doneCreating k8s_node_1   ... 
Creating k8s_node_2   ... 
Creating k8s_node_3   ... 
ting k8s_node_2   ... done

In [12]:
!cp kubeconfig.yaml ~/.kube/config

In [13]:
cat kubeconfig.yaml

apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUM1RENDQWN5Z0F3SUJBZ0lCQURBTkJna3Foa2lHOXcwQkFRc0ZBREFqTVJBd0RnWURWUVFLRXdkck0zTXQKYjNKbk1ROHdEUVlEVlFRREV3WnJNM010WTJFd0hoY05NVGt3TmpFMk1UazFNREU1V2hjTk1qa3dOakV6TVRrMQpNREU1V2pBak1SQXdEZ1lEVlFRS0V3ZHJNM010YjNKbk1ROHdEUVlEVlFRREV3WnJNM010WTJFd2dnRWlNQTBHCkNTcUdTSWIzRFFFQkFRVUFBNElCRHdBd2dnRUtBb0lCQVFER3JFalBmdk5kdVRIWGwzZkFHNmRNKzk0eEhuZncKUUxaQ2lCOXZNRmFPb1d6NWFzSzZ3MjJOQnc2QkJ4bnk2a3dyYjd0cXdXTXhOVHlNNWdHdjk5N1hFZGRPTWxLUQpFN1liYWRFYUpYdEorSVdxYlV6MHR3b21GTHJFOHZBbzRrZFEyU2NnU2JSUXBJSUc5UDhSLzVqQ1FFZkNIV1pYCkNhakg1WmhsTmxiYWp5VjdYc3RZRE95Q0FCZ2lvRWRkM0J1cDNtaGJGUXViYTZxd3RhRXBQaXRMQmVXNkduelQKcFMxS01ZZkhydnVYZU1JTWhFUDFSankwazYzcnN0TG05Z0h3MzExa21mTFV1Ryt5YXVrSUJTS3RHMmRiL1I2MApaUGpZOUU5ZnBRaVEzaGlJZ2JSSjFHQ2FXdWN0bHdCRmZ0eWpucE9RU1UyVXZ3d1daWTBuTzhWaEFnTUJBQUdqCkl6QWhNQTRHQTFVZER3RUIvd1FFQXdJQ3BEQVBCZ05WSFJNQkFmOEVCVEFEQVFIL01BMEdDU3FHU0liM0RRRUIKQ3dVQUE0SUJBUUFlOU50bkJZalp3

In [14]:
!kubectl get pods

No resources found.


In [15]:

!kubectl run hello-world \
    --replicas=2 \
    --labels="run=load-balancer-example" \
    --image=gcr.io/google-samples/node-hello:1.0  \
    --port=8080

deployment.apps "hello-world" created


In [19]:
!kubectl get deployments

NAME          DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
hello-world   2         2         2            2           3m


In [25]:
!kubectl describe deployments

Name:                   hello-world
Namespace:              default
CreationTimestamp:      Sun, 16 Jun 2019 22:01:29 +0200
Labels:                 run=load-balancer-example
Annotations:            deployment.kubernetes.io/revision=1
Selector:               run=load-balancer-example
Replicas:               2 desired | 2 updated | 2 total | 2 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  1 max unavailable, 1 max surge
Pod Template:
  Labels:  run=load-balancer-example
  Containers:
   hello-world:
    Image:        gcr.io/google-samples/node-hello:1.0
    Port:         8080/TCP
    Host Port:    0/TCP
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Available      True    MinimumReplicasAvailable
OldReplicaSets:  <none>
NewReplicaSet:   hello-world-68ff65cf7 (2/2 replicas created)
Events:
  Type    Reason             

In [26]:
!kubectl get pods --selector="run=load-balancer-example" --output=wide

NAME                          READY     STATUS    RESTARTS   AGE       IP          NODE
hello-world-68ff65cf7-8lbzg   1/1       Running   0          4m        10.42.2.4   b7d984be7952
hello-world-68ff65cf7-wzwgr   1/1       Running   0          4m        10.42.1.4   99539420d0d2


### rescale deployment to make distribution more obvious

In [32]:
!kubectl scale deployment --replicas=10 "hello-world"

deployment.extensions "hello-world" scaled


In [34]:
!kubectl get deployments

NAME          DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
hello-world   10        10        10           7           8m


In [37]:
!kubectl get pods --selector="run=load-balancer-example" --output=wide

NAME                          READY     STATUS              RESTARTS   AGE       IP          NODE
hello-world-68ff65cf7-85552   1/1       Running             0          55s       10.42.2.5   b7d984be7952
hello-world-68ff65cf7-8lbzg   1/1       Running             0          9m        10.42.2.4   b7d984be7952
hello-world-68ff65cf7-9bzzh   0/1       ContainerCreating   0          55s       <none>      05c089864933
hello-world-68ff65cf7-g9qgn   0/1       ContainerCreating   0          1m        <none>      05c089864933
hello-world-68ff65cf7-h2g5r   1/1       Running             0          55s       10.42.2.6   b7d984be7952
hello-world-68ff65cf7-jwmdc   1/1       Running             0          55s       10.42.1.5   99539420d0d2
hello-world-68ff65cf7-k8zj4   1/1       Running             0          55s       10.42.1.6   99539420d0d2
hello-world-68ff65cf7-pzhdm   1/1       Running             0          55s       10.42.1.7   99539420d0d2
hello-world-68ff65cf7-rdgwv   0/1       ContainerCreat

In [46]:
!kubectl get nodes --output=wide

NAME           STATUS    ROLES     AGE       VERSION         EXTERNAL-IP   OS-IMAGE   KERNEL-VERSION     CONTAINER-RUNTIME
05c089864933   Ready     <none>    25m       v1.14.1-k3s.4   <none>        Unknown    4.9.125-linuxkit   containerd://1.2.5+unknown
99539420d0d2   Ready     <none>    25m       v1.14.1-k3s.4   <none>        Unknown    4.9.125-linuxkit   containerd://1.2.5+unknown
b7d984be7952   Ready     <none>    25m       v1.14.1-k3s.4   <none>        Unknown    4.9.125-linuxkit   containerd://1.2.5+unknown


### Let's kill some nodes in a non graceful way (e.g. physical outage or other nasty stuff)

In [47]:
!docker-compose up -d --scale node=1

Stopping and removing k8s_node_2 ... 
Stopping and removing k8s_node_3 ... 
ping and removing k8s_node_3 ... done

In [49]:
!docker-compose ps

    Name                 Command             State              Ports           
--------------------------------------------------------------------------------
k8s_node_1     /bin/k3s agent                Up                                 
k8s_server_1   /bin/k3s server --disable-    Up      0.0.0.0:6443->6443/tcp,    
               ...                                   0.0.0.0:8888->80/tcp,      
                                                     0.0.0.0:8880->8880/tcp     


In [50]:
!kubectl get nodes --output=wide

NAME           STATUS     ROLES     AGE       VERSION         EXTERNAL-IP   OS-IMAGE   KERNEL-VERSION     CONTAINER-RUNTIME
05c089864933   Ready      <none>    26m       v1.14.1-k3s.4   <none>        Unknown    4.9.125-linuxkit   containerd://1.2.5+unknown
99539420d0d2   NotReady   <none>    26m       v1.14.1-k3s.4   <none>        Unknown    4.9.125-linuxkit   containerd://1.2.5+unknown
b7d984be7952   NotReady   <none>    26m       v1.14.1-k3s.4   <none>        Unknown    4.9.125-linuxkit   containerd://1.2.5+unknown


In [54]:
!kubectl get pods --selector="run=load-balancer-example" --output=wide

NAME                          READY     STATUS    RESTARTS   AGE       IP          NODE
hello-world-68ff65cf7-85552   1/1       Running   0          11m       10.42.2.5   b7d984be7952
hello-world-68ff65cf7-8lbzg   1/1       Running   0          19m       10.42.2.4   b7d984be7952
hello-world-68ff65cf7-9bzzh   1/1       Running   0          11m       10.42.0.6   05c089864933
hello-world-68ff65cf7-g9qgn   1/1       Running   0          11m       10.42.0.4   05c089864933
hello-world-68ff65cf7-h2g5r   1/1       Running   0          11m       10.42.2.6   b7d984be7952
hello-world-68ff65cf7-jwmdc   1/1       Running   0          11m       10.42.1.5   99539420d0d2
hello-world-68ff65cf7-k8zj4   1/1       Running   0          11m       10.42.1.6   99539420d0d2
hello-world-68ff65cf7-pzhdm   1/1       Running   0          11m       10.42.1.7   99539420d0d2
hello-world-68ff65cf7-rdgwv   1/1       Running   0          11m       10.42.0.5   05c089864933
hello-world-68ff65cf7-wzwgr   1/1       Running 

K8s does not care much and still things that pods are running and alive even if nodes are "not ready"

In [53]:
!kubectl get nodes --output=wide

NAME           STATUS     ROLES     AGE       VERSION         EXTERNAL-IP   OS-IMAGE   KERNEL-VERSION     CONTAINER-RUNTIME
05c089864933   Ready      <none>    30m       v1.14.1-k3s.4   <none>        Unknown    4.9.125-linuxkit   containerd://1.2.5+unknown
99539420d0d2   NotReady   <none>    30m       v1.14.1-k3s.4   <none>        Unknown    4.9.125-linuxkit   containerd://1.2.5+unknown
b7d984be7952   NotReady   <none>    30m       v1.14.1-k3s.4   <none>        Unknown    4.9.125-linuxkit   containerd://1.2.5+unknown


Drain is a suggestion for K8s to stop scheduling anything to those nodes.
forced one should work but I did not care

In [56]:
!kubectl drain b7d984be7952

node "b7d984be7952" cordoned
error: unable to drain node "b7d984be7952", aborting command...

There are pending nodes to be drained:
 b7d984be7952
error: DaemonSet-managed pods (use --ignore-daemonsets to ignore): svclb-traefik-njwbm


In [57]:
!docker-compose up -d --scale node=3

k8s_server_1 is up-to-date
Starting k8s_node_1 ... 
Creating k8s_node_2 ... mdone
Creating k8s_node_3 ... 
ting k8s_node_2 ... done

In [61]:
!kubectl get nodes --output=wide

NAME           STATUS                        ROLES     AGE       VERSION         EXTERNAL-IP   OS-IMAGE   KERNEL-VERSION     CONTAINER-RUNTIME
05c089864933   Ready                         <none>    32m       v1.14.1-k3s.4   <none>        Unknown    4.9.125-linuxkit   containerd://1.2.5+unknown
99539420d0d2   NotReady,SchedulingDisabled   <none>    32m       v1.14.1-k3s.4   <none>        Unknown    4.9.125-linuxkit   containerd://1.2.5+unknown
a0d0e78fb44c   Ready                         <none>    31s       v1.14.1-k3s.4   <none>        Unknown    4.9.125-linuxkit   containerd://1.2.5+unknown
b7d984be7952   NotReady,SchedulingDisabled   <none>    32m       v1.14.1-k3s.4   <none>        Unknown    4.9.125-linuxkit   containerd://1.2.5+unknown
cb1d24640ecb   Ready                         <none>    31s       v1.14.1-k3s.4   <none>        Unknown    4.9.125-linuxkit   containerd://1.2.5+unknown


We have added 2 new nodes as a "replacement" for the killed ones. Even though they have same name for docker, these are new for K8s

In [60]:
# drain and delete "not ready" nodes
!kubectl drain 99539420d0d2

node "99539420d0d2" cordoned
error: unable to drain node "99539420d0d2", aborting command...

There are pending nodes to be drained:
 99539420d0d2
error: DaemonSet-managed pods (use --ignore-daemonsets to ignore): svclb-traefik-tlb5k


In [63]:
!kubectl delete node 99539420d0d2

node "99539420d0d2" deleted


In [64]:
!kubectl delete node b7d984be7952

node "b7d984be7952" deleted


In [65]:
!kubectl get nodes --output=wide

NAME           STATUS    ROLES     AGE       VERSION         EXTERNAL-IP   OS-IMAGE   KERNEL-VERSION     CONTAINER-RUNTIME
05c089864933   Ready     <none>    34m       v1.14.1-k3s.4   <none>        Unknown    4.9.125-linuxkit   containerd://1.2.5+unknown
a0d0e78fb44c   Ready     <none>    2m        v1.14.1-k3s.4   <none>        Unknown    4.9.125-linuxkit   containerd://1.2.5+unknown
cb1d24640ecb   Ready     <none>    2m        v1.14.1-k3s.4   <none>        Unknown    4.9.125-linuxkit   containerd://1.2.5+unknown


while we were adding nodes, K8s moved all pods to the single a single node. 

In [69]:
!kubectl scale deployment --replicas=10 "hello-world"

deployment.extensions "hello-world" scaled


In [72]:
!kubectl get pods --selector="run=load-balancer-example" --output=wide

NAME                          READY     STATUS    RESTARTS   AGE       IP           NODE
hello-world-68ff65cf7-5tsrz   1/1       Running   0          5m        10.42.0.11   05c089864933
hello-world-68ff65cf7-94kjs   1/1       Running   0          5m        10.42.0.9    05c089864933
hello-world-68ff65cf7-9bzzh   1/1       Running   0          17m       10.42.0.6    05c089864933
hello-world-68ff65cf7-b5b87   1/1       Running   0          5m        10.42.0.12   05c089864933
hello-world-68ff65cf7-bf6zb   1/1       Running   0          5m        10.42.0.13   05c089864933
hello-world-68ff65cf7-frtqq   1/1       Running   0          5m        10.42.0.7    05c089864933
hello-world-68ff65cf7-g9qgn   1/1       Running   0          17m       10.42.0.4    05c089864933
hello-world-68ff65cf7-lznlw   1/1       Running   0          5m        10.42.0.14   05c089864933
hello-world-68ff65cf7-rdgwv   1/1       Running   0          17m       10.42.0.5    05c089864933
hello-world-68ff65cf7-x84nr   1/1     

Let's try to force K8 to redistribute pods across all available nodes

In [71]:
!kubectl get deployments

NAME          DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
hello-world   10        10        10           10          25m


In [73]:
!kubectl scale deployment --replicas=9 "hello-world"

deployment.extensions "hello-world" scaled


In [74]:
!kubectl scale deployment --replicas=10 "hello-world"

deployment.extensions "hello-world" scaled


In [75]:
!kubectl get pods --selector="run=load-balancer-example" --output=wide

NAME                          READY     STATUS              RESTARTS   AGE       IP           NODE
hello-world-68ff65cf7-5tsrz   1/1       Running             0          6m        10.42.0.11   05c089864933
hello-world-68ff65cf7-7cx62   0/1       ContainerCreating   0          7s        <none>       a0d0e78fb44c
hello-world-68ff65cf7-94kjs   1/1       Running             0          6m        10.42.0.9    05c089864933
hello-world-68ff65cf7-9bzzh   1/1       Running             0          17m       10.42.0.6    05c089864933
hello-world-68ff65cf7-b5b87   1/1       Terminating         0          6m        10.42.0.12   05c089864933
hello-world-68ff65cf7-bf6zb   1/1       Running             0          6m        10.42.0.13   05c089864933
hello-world-68ff65cf7-frtqq   1/1       Running             0          6m        10.42.0.7    05c089864933
hello-world-68ff65cf7-g9qgn   1/1       Running             0          17m       10.42.0.4    05c089864933
hello-world-68ff65cf7-lznlw   1/1       Runni

In [77]:
!kubectl get pods --selector="run=load-balancer-example" --output=wide

NAME                          READY     STATUS    RESTARTS   AGE       IP           NODE
hello-world-68ff65cf7-5tsrz   1/1       Running   0          16m       10.42.0.11   05c089864933
hello-world-68ff65cf7-7cx62   1/1       Running   0          10m       10.42.3.3    a0d0e78fb44c
hello-world-68ff65cf7-94kjs   1/1       Running   0          16m       10.42.0.9    05c089864933
hello-world-68ff65cf7-9bzzh   1/1       Running   0          28m       10.42.0.6    05c089864933
hello-world-68ff65cf7-bf6zb   1/1       Running   0          16m       10.42.0.13   05c089864933
hello-world-68ff65cf7-frtqq   1/1       Running   0          16m       10.42.0.7    05c089864933
hello-world-68ff65cf7-g9qgn   1/1       Running   0          28m       10.42.0.4    05c089864933
hello-world-68ff65cf7-lznlw   1/1       Running   0          16m       10.42.0.14   05c089864933
hello-world-68ff65cf7-rdgwv   1/1       Running   0          28m       10.42.0.5    05c089864933
hello-world-68ff65cf7-x84nr   1/1     

a trick with scale down/up worked

In [78]:
!kubectl scale deployment --replicas=3 "hello-world"

deployment.extensions "hello-world" scaled


In [79]:
!kubectl scale deployment --replicas=10 "hello-world"

deployment.extensions "hello-world" scaled


In [83]:
!kubectl get pods --selector="run=load-balancer-example" --output=wide

NAME                          READY     STATUS              RESTARTS   AGE       IP          NODE
hello-world-68ff65cf7-47czf   0/1       ContainerCreating   0          37s       <none>      cb1d24640ecb
hello-world-68ff65cf7-4zw9j   1/1       Running             0          37s       10.42.3.6   a0d0e78fb44c
hello-world-68ff65cf7-6vj86   1/1       Running             0          37s       10.42.3.5   a0d0e78fb44c
hello-world-68ff65cf7-9bzzh   1/1       Running             0          29m       10.42.0.6   05c089864933
hello-world-68ff65cf7-g9qgn   1/1       Running             0          29m       10.42.0.4   05c089864933
hello-world-68ff65cf7-jss5c   0/1       ContainerCreating   0          37s       <none>      cb1d24640ecb
hello-world-68ff65cf7-l2ttq   0/1       ContainerCreating   0          37s       <none>      cb1d24640ecb
hello-world-68ff65cf7-n6gpz   1/1       Running             0          37s       10.42.3.7   a0d0e78fb44c
hello-world-68ff65cf7-rdgwv   1/1       Running       

K8s should redistribute pods by itself depending on how busy they are, but trick above might work when some manual intervention is requried. 

For automatic control of this behaviour https://github.com/kubernetes-incubator/descheduler could be used and/or https://kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale/

In [84]:
!kubectl delete deployment "hello-world"

deployment.extensions "hello-world" deleted


In [89]:
!kubectl get pods --selector="run=load-balancer-example" --output=wide

NAME                          READY     STATUS        RESTARTS   AGE       IP          NODE
hello-world-68ff65cf7-4zw9j   1/1       Terminating   0          1m        10.42.3.6   a0d0e78fb44c
hello-world-68ff65cf7-6vj86   1/1       Terminating   0          1m        10.42.3.5   a0d0e78fb44c
hello-world-68ff65cf7-9bzzh   1/1       Terminating   0          29m       10.42.0.6   05c089864933
hello-world-68ff65cf7-g9qgn   1/1       Terminating   0          30m       10.42.0.4   05c089864933
hello-world-68ff65cf7-n6gpz   1/1       Terminating   0          1m        10.42.3.7   a0d0e78fb44c
hello-world-68ff65cf7-rdgwv   1/1       Terminating   0          29m       10.42.0.5   05c089864933
hello-world-68ff65cf7-thftf   1/1       Terminating   0          1m        10.42.3.4   a0d0e78fb44c
